# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
#Импортируем библиотеки, создаем датафрейм из файла, смотрим первые 5 строк таблицы
import pandas as pd
from pymystem3 import Mystem
data = pd.read_csv('/datasets/data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [ ]:
#Просматриваем уникальные значения каждого столбца и их количество
for column in data:
    print(data[column].value_counts(dropna=False))

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
 NaN            2174
-1645.463049       1
-6620.396473       1
-1238.560080       1
-3047.519891       1
                ... 
-2849.351119       1
-5619.328204       1
-448.829898        1
-1687.038672       1
-206.107342        1
Name: days_employed, Length: 19352, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: d

In [ ]:
# КОД РЕВЬЮВЕРА

data.describe()

#Ох, здорово, спасибо

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

При изучении датасета были замечены проблемы:
- в столбцах days_employed и total_income присутствуют отрицательные вещественные числа, а так же NaN-значения.
- в столбце children присутствует отрицательное значение, а так же 76 человек имеют по 20 детей, что выбивается из остального списка.
- dob_years имеет нулевые значения
- строки в столбцеах education и family_status отличаются регистром
- цели кредита в столбце purpose написаны вразнобой

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
#заполняем пропуски в столбце зарплат медианными значениями типов занятости
med_salary = data.groupby('income_type')['total_income'].transform('median')
data['total_income'] = data['total_income'].fillna(med_salary)
#функция переводит часы в дни у соответствующих значений столбца с рабочими днями
def hours_to_days(row):
    if row > 320000:
        return row / 24
    return row
data['days_employed'] = data['days_employed'].apply(hours_to_days)

#заполняем пропуски в столбце рабочих дней медианными значениями типов занятости
med_days = data.groupby('income_type')['days_employed'].transform('median')
data['days_employed'] = data['days_employed'].fillna(med_days)

#меняем неопределенное значение пола на самое распространенное в датасете
data.replace('XNA', 'F', inplace = True)

#заменяем нулевые значения возраста на медианные значения различных семейных статусов
med_age = data.groupby('family_status')['dob_years'].transform('median')
data['dob_years'].replace(0, med_age, inplace = True)

data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,851.546834,43.492869,0.817236,0.972544,0.080883,1.652253e+05
std,1.381587,7007.842448,12.222265,0.548138,1.420324,0.272661,9.804367e+04
min,-1.000000,-18388.949901,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2570.047544,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1355.683356,43.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,-316.240646,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,20.000000,16739.808353,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Нулевой доход у пенсионеров и студентов возможен в реальности, в остальных случаях люди либо предпочли не указывать доход, либо произошла ошибка при записи в датасет.

### Замена типа данных

In [ ]:
#Меняем float на int и переводим отрицательные числа в положительные
data['days_employed'] = data['days_employed'].astype('int').abs()
data['total_income'] = data['total_income'].astype('int')
data['children'] = data['children'].abs()
#здесь же заодно обрабатываем выброс в 20 детей у нескольких людей
data['children'].replace(20, 2, inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Проверка показала успешную замену типов данных

### Обработка дубликатов

In [ ]:
#Просматриваем количество дубликатов в датасете
data.duplicated().sum()
#Переводим строки в нижний регистр, в столбце education это избавит от дубликатов,
#в столбце family_status приведет написание к одному стандарту
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
#удаляем дубликаты
data = data.drop_duplicates().reset_index(drop = True)
#проверяем
data.duplicated().sum()

0

**Вывод**

Возможно, датасет составлялся из нескольких таблиц, в которых данные записывались по-разному. Это могло послужить причиной появления строк в разном регистре.

### Лемматизация

In [ ]:
# Создаем функицю, которая лемматизирует строки в полученном столбце и проверяет
# совпадает ли слово с строке одному из слов в списке
# если да, то данная строка заменяется на совпадающее слово
# если ни одно слово не совпадает со списком, вылезет ошибка, которая заставит расширить список целей
m = Mystem()

def lemmas(row):
    lemma = m.lemmatize(row)
    purposes = ["жилье", "недвижимость", "образование", "автомобиль", "свадьба"]
    for word in lemma:
        if word in purposes:
            return word
    raise ValueError('Цель не из списка')
# Проверяем цели
data['purpose'] = data['purpose'].apply(lemmas)
data['purpose'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2323
Name: purpose, dtype: int64

**Вывод**

Лемматизация показала, что 50 процентов клиентов берут кредит для операций с недвижимостью и жильем

### Категоризация данных

**Вывод**

Категоризацию я сделал в последующих шагах, там, где это требовалось для ответа на вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#создаем словарь из данных по детям и долгам
children_data = data[['children', 'debt']].copy()
#создаем функцию, которая опеределяет есть ли дети у человека или нет
def have_children(row):
    if row == 0:
        return 'Нет детей'
    if 1 <= row <= 2:
        return 'Малодетная семья'
    if row >= 3:
        return 'Многодетная семья'
#создаем новый столбец для категоризации людей по наличию детей с применением новой функции
children_data['have_children'] = children_data['children'].apply(have_children)
#вычисляем долю должников с детьми и без из общего количества людей с детьми и без
children_data.groupby('have_children')['debt'].sum() / children_data.groupby('have_children')['debt'].count()

have_children
Малодетная семья     0.092654
Многодетная семья    0.081579
Нет детей            0.075444
Name: debt, dtype: float64

**Вывод**

Люди, в семье которых от 1 до 2 детей, чаще остальных задерживают выплаты

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#вычисляем  долю должников из общего количества людей с группировкой по семейному статусу
data.groupby('family_status')['debt'].sum() / data.groupby('family_status')['debt'].count()

family_status
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093494
женат / замужем          0.075452
не женат / не замужем    0.097509
Name: debt, dtype: float64

**Вывод**

Холостяки и люди в гражданском браке чаще всего задержвают выплаты по кредитам

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#создаем словарь с данными о доходах и долгах
income_data = data[['total_income', 'debt']].copy()
#создаем новый столбец, где делим доходы клиентов на 4 квартиля и присваиваем каждому квартилю название
income_data['income_labels'] = pd.qcut(income_data['total_income'], 4, labels=['низкий', 'средний', 'выше среднего', 'высокий'])
#вычисляем долю должников
income_data.groupby('income_labels')['debt'].sum() / income_data.groupby('income_labels')['debt'].count()

income_labels
низкий           0.079605
средний          0.088171
выше среднего    0.085366
высокий          0.071415
Name: debt, dtype: float64

**Вывод**

Люди со средним доходом чаще всего задерживают выплаты

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#вычисляем  долю должников из общего количества людей с группировкой по целям кредита
data.groupby('purpose')['debt'].sum() / data.groupby('purpose')['debt'].count()

purpose
автомобиль      0.093590
жилье           0.069058
недвижимость    0.074634
образование     0.092200
свадьба         0.080069
Name: debt, dtype: float64

**Вывод**

Люди, берущие кредит на авто и образование, чаще других задержвают выплаты по кредитам

## Шаг 4. Общий вывод

Отвечая на отдельные вопросы, делать какие-то определенные выводы о вероятности возврата кредита в срок нельзя. Все эти факторы должны рассматриваться вместе, потому что, как выяснилось, даже люди с огромным доходом часто задерживают выплаты, в то время, как люди с детьми - нет, хотя ребенок является огромной финансовой нагрузкой.